station_data_aggregation
---
This script reads information of stations and create a shapefile.\
Donghoon Lee @ Oct-7-2019


In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [2]:
filn_in = './stations.xlsx'
filn_out = './stations.shp'

# Read information of stations
df = pd.read_excel(filn_in)
# Trim columns
df = df[df.columns[2:]]
# Remain available stations
df = df[~df['CODIGO'].isna()]

# Create a shapefile of stations
df[['Lon','Lat']] = df[['Lon','Lat']].apply(pd.to_numeric)
df['Coordinates'] = list(zip(df.Lon, df.Lat))           # Coordinates
df['Coordinates'] = df['Coordinates'].apply(Point)      # tuples to Shapely's Point
crs = {'init': 'epsg:4326'}
gdf = gpd.GeoDataFrame(df, crs=crs, geometry='Coordinates')
gdf.to_file(filn_out)
print('%s is saved.' % filn_out)

./stations.shp is saved.


In [3]:
gdf.head()


,Region,Type,CODIGO,Name,Lat,Lon,Altitude,Monitoring,Note,Same station,Coordinates
0,Piura,Meteorology,104016,LANCONES,-4.642880,-80.547170,133.0,Conventional,Out of the basin,472FD6CE,POINT (-80.54716999999999 -4.64288)
1,Piura,Meteorology,472FD6CE,LANCONES,-4.642878,-80.547175,133.0,Automatic,Out of the basin,104016,POINT (-80.547175 -4.6428778)
2,Piura,Meteorology,105105,CHUSIS,-5.527560,-80.812750,8.0,Conventional,In the basin,NaN,POINT (-80.81274999999999 -5.52756)
3,Piura,Meteorology,105012,BERNAL,-5.454500,-80.742620,14.0,Conventional,In the basin,NaN,POINT (-80.74262 -5.4545)
4,Piura,Meteorology,104090,LA ESPERANZA,-4.921140,-81.059690,7.0,Conventional,Out of the basin,NaN,POINT (-81.05969 -4.92114)


In [4]:
from ipyleaflet import *
m = Map(center=(52, 10), zoom=8, basemap=basemaps.Hydda.Full)
m

Map(basemap={'url': 'http://{s}.tile.openstreetmap.se/hydda/full/{z}/{x}/{y}.png', 'max_zoom': 18, 'attributio…

In [5]:
strata_all = basemap_to_tiles(basemaps.Strava.All)
m.add_layer(strata_all)

In [6]:
from ipywidgets import Label

label = Label()
display(label)

def handle_interaction(**kwargs):
    if kwargs.get('type') == 'mousemove':
        label.value = str(kwargs.get('coordinates'))

m.on_interaction(handle_interaction)

Label(value='')

In [7]:
strata_water = basemap_to_tiles(basemaps.Strava.Water)
m.substitute_layer(strata_all, strata_water)

In [9]:
# Adding an overlay layer
import json

with open('./europe_110.geo.json') as f:
    data = json.load(f)
for feature in data['features']:
    feature['properties']['style'] = {
        'color': 'grey',
        'weight': 1,
        'fillColor': 'grey',
        'fillOpacity': 0.5
    }
geo = GeoJSON(data=data, hover_style={'fillColor': 'red'}, name='Countries')
m.add_layer(geo)

In [10]:
# Adding the control
m.add_control(LayersControl())

In [11]:
# Creating two maps side by side
import ipywidgets
 
ipywidgets.HBox([m, Map(center=[43.6, 1.44], zoom=10)])